In [20]:
%run data_preprocessing.ipynb

19
10
13
18
20
21
9
{0: [13, 7, 13, 14, 13, 10, 5, 4, 3, 2, 2, 0, 4, 12, 13, 14, 5, 3, 10, 9, 14, 10, 1, 2, 4, 7, 26, 4, 12, 12, 23, 22, 27, 10, 12, 17, 44, 3, 4, 19, 7, 16, 9, 42, 57, 14, 14, 18, 17, 16, 18, 23, 4, 19, 5, 2, 18, 8, 6, 6, 3, 5, 4, 4, 4, 4, 9, 21, 19, 5, 19, 16, 33, 11, 11, 11, 66, 11, 15, 5, 13, 14, 4, 4, 3, 1, 3, 1, 3, 2, 4, 1, 0, 21, 20, 20, 20, 39, 12, 11, 5, 4, 22, 13, 13, 13, 15, 8, 11, 8, 12, 16, 16, 7, 4, 5, 4, 5, 5, 6, 2, 3, 7, 14, 16, 8, 8, 14, 8, 10, 11, 9, 150, 9, 7, 10, 14, 12, 166, 6, 5, 11, 1, 5, 6, 7, 4, 1, 2, 2, 1, 6, 0, 16, 14, 9, 19, 11, 8, 9, 21, 33, 31, 30, 65, 73, 45, 64, 61, 90, 66, 51, 26, 15, 22, 20, 12, 7, 7, 10, 8, 8, 99, 59, 55, 43, 21, 14, 21, 26, 25, 26, 26, 38, 38, 35, 24, 15, 14, 18, 11, 12, 11, 8, 17, 16, 21, 12, 8, 20, 47, 66, 51, 67, 46, 48, 11, 14, 12, 18, 15, 25, 20, 17, 24, 15, 20, 16, 17, 11, 9, 16, 15, 12, 14, 17, 20, 18, 16, 14, 17, 14, 65, 63, 37, 57, 44, 28, 9, 5, 7, 5, 8, 5, 6, 5, 11, 10, 9, 6, 7, 10, 11, 10, 8, 0, 5, 6, 9, 7,

[([13, 7, 13, 14, 13, 10, 5, 4, 3, 2], (0, 0, 0)), ([7, 13, 14, 13, 10, 5, 4, 3, 2, 2], (0, 4, 0)), ([13, 14, 13, 10, 5, 4, 3, 2, 2, 0], (0, 12, 1)), ([14, 13, 10, 5, 4, 3, 2, 2, 0, 4], (0, 13, 1)), ([13, 10, 5, 4, 3, 2, 2, 0, 4, 12], (0, 14, 1)), ([10, 5, 4, 3, 2, 2, 0, 4, 12, 13], (0, 5, 0)), ([5, 4, 3, 2, 2, 0, 4, 12, 13, 14], (0, 3, 0)), ([4, 3, 2, 2, 0, 4, 12, 13, 14, 5], (0, 10, 1)), ([3, 2, 2, 0, 4, 12, 13, 14, 5, 3], (0, 9, 1)), ([2, 2, 0, 4, 12, 13, 14, 5, 3, 10], (0, 14, 1)), ([2, 0, 4, 12, 13, 14, 5, 3, 10, 9], (0, 10, 1)), ([0, 4, 12, 13, 14, 5, 3, 10, 9, 14], (0, 1, 0)), ([4, 12, 13, 14, 5, 3, 10, 9, 14, 10], (0, 2, 0)), ([12, 13, 14, 5, 3, 10, 9, 14, 10, 1], (0, 4, 0)), ([13, 14, 5, 3, 10, 9, 14, 10, 1, 2], (0, 7, 1)), ([14, 5, 3, 10, 9, 14, 10, 1, 2, 4], (0, 26, 2)), ([5, 3, 10, 9, 14, 10, 1, 2, 4, 7], (0, 4, 0)), ([3, 10, 9, 14, 10, 1, 2, 4, 7, 26], (0, 12, 1)), ([10, 9, 14, 10, 1, 2, 4, 7, 26, 4], (0, 12, 1)), ([9, 14, 10, 1, 2, 4, 7, 26, 4, 12], (0, 23, 2)), ([14, 10,

38913


In [21]:
import torch
from transformers import BertModel
from simpletransformers.classification import ClassificationModel
import logging
from sklearn.metrics import f1_score, accuracy_score

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [22]:
summary_state = summary_state.drop(columns=["State", "Count"])
summary_state = summary_state.rename(columns={"Summary":"text", "Label":"labels"})
summary_state = summary_state.reset_index(drop=True)

In [23]:
separation = int(0.8 * len(summary_state))
train_data = summary_state[:separation]
test_data = summary_state[separation:]
print(len(test_data))

train_data.iloc[13]

32


text      Benin: 1 District in alert phase ( Ouake) Burk...
labels                                                    2
Name: 13, dtype: object

In [24]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=3, 
                            use_cuda=False, args={"train_batch_size": 1 ,
                                                  'reprocess_input_data': True, 'overwrite_output_dir': True})


In [25]:
model.train_model(train_data)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


Running loss: 1.462109

/Users/amirgirgis/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.004221


/Users/amirgirgis/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [26]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test_data, f1=f1_multiclass, acc=accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/Users/amirgirgis/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'f1': 0.875, 'acc': 0.875, 'eval_loss': 0.8242821748135611}


In [ ]:
print(model_outputs)